# Portfolio Greeks

The file `etf_holdings.csv` contains the options holdings for a variety of ETFs.  In this assignment you will calculate the net greeks for each of the ETFs.  Here is some guidance:

1. Use the **bizdays** package to calculate the number of business days to expiration.
1. Assume a risk-free rate of 0.05 and dividend yield of 0.0075.
1. Use the **py_vollib** package to calculate the implied volatility and greeks: delta, gamma, theta, vega.
1. There are some deep in the money options for which the `implied_volatility()` function breaks.  You will need to figure out a work around for this.
1. Greeks are additive, so you will be able to use a simple `.groupby('Account')` on the individual option greeks to come up with the portfolio greeks.

In [1]:
import pandas as pd
import numpy as np
import bizdays
from py_vollib.black_scholes import black_scholes

# Load the data (assuming you've already converted the image data into a CSV or dataframe)
data = pd.read_csv('/Users/yuanhanlim/Desktop/DS & ML/etf_holdings.csv')

# Define some constants
risk_free_rate = 0.05
dividend_yield = 0.0075

# Create a business calendar that excludes weekends
cal = bizdays.Calendar(weekdays=['saturday', 'sunday'])

# Calculate business days to expiration for each row
def biz_days_to_expiration(row):
    return cal.bizdays(row['Date'], row['MaturityDate'])

# Apply the function to create a column for business days to expiration
data['BizDaysToExp'] = data.apply(biz_days_to_expiration, axis=1)

/var/folders/79/6xh3y3cd7kl6qqpcykmzhz8r0000gn/T/ipykernel_12883/1466138509.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from py_vollib.black_scholes_merton.implied_volatility import implied_volatility as bsm_implied_volatility

# Function to calculate implied volatility with dividend yield using Black-Scholes-Merton
def calculate_implied_vol_with_dividend(row):
    try:
        # 'C' for call, 'P' for put, based on the Type in your dataset
        return bsm_implied_volatility(
            price=row['Price'],                        # Option price
            S=row['UPX'],                              # Underlying price (SPY price in your data)
            K=row['StrikePrice'],                      # Strike price
            t=row['BizDaysToExp'] / 252,               # Time to expiration in years
            r=risk_free_rate,                          # Risk-free interest rate
            flag=row['Type'].lower(),                  # 'c' for call, 'p' for put
            q=dividend_yield                           # Dividend yield (continuous)
        )
    except Exception as e:
        return 5  # Assume 500% for problematic options

# Apply the function to calculate implied volatility with dividends
data['ImpliedVol'] = data.apply(calculate_implied_vol_with_dividend, axis=1)

In [3]:
from py_vollib.black_scholes_merton.greeks import analytical as bsm_greeks

def calculate_greeks(row):
    try:
        # Extract relevant data from row
        S = row['UPX']  # Underlying price
        K = row['StrikePrice']  # Strike price
        t = row['BizDaysToExp'] / 252  # Time to expiration (convert to years)
        r = 0.05  # Risk-free rate
        q = 0.0075  # Dividend yield
        sigma = row['ImpliedVol']  # Implied volatility
        flag = row['Type'].lower()  # 'c' for call, 'p' for put

        # Calculate Greeks using Black-Scholes-Merton
        delta = bsm_greeks.delta(flag, S, K, t, r, sigma, q)
        gamma = bsm_greeks.gamma(flag, S, K, t, r, sigma, q)
        theta = bsm_greeks.theta(flag, S, K, t, r, sigma, q)
        vega = bsm_greeks.vega(flag, S, K, t, r, sigma, q)

        return pd.Series({'Delta': delta, 'Gamma': gamma, 'Theta': theta, 'Vega': vega})

    except Exception as e:
        return pd.Series({'Delta': np.nan, 'Gamma': np.nan, 'Theta': np.nan, 'Vega': np.nan})

# Apply the function to calculate Greeks for each row in your data
data[['Delta', 'Gamma', 'Theta', 'Vega']] = data.apply(calculate_greeks, axis=1)

In [4]:
# Multiply by the number of shares for each option
data['WeightedDelta'] = data['Delta'] * data['Shares']
data['WeightedGamma'] = data['Gamma'] * data['Shares']
data['WeightedTheta'] = data['Theta'] * data['Shares']
data['WeightedVega'] = data['Vega'] * data['Shares']

# Group by account to get the portfolio-level greeks
portfolio_greeks = data.groupby('Account').agg({
    'WeightedDelta': 'sum',
    'WeightedGamma': 'sum',
    'WeightedTheta': 'sum',
    'WeightedVega': 'sum'
})

portfolio_greeks


,WeightedDelta,WeightedGamma,WeightedTheta,WeightedVega
Account,,,,
APRT,0.657679,-0.001538,0.045581,-0.529693
APRW,0.450502,-0.001281,0.061697,-0.522490
ARLU,0.684541,0.004191,-0.023241,0.950377
AUGT,0.616810,-0.001645,0.055104,-0.893075
AUGU,0.625639,0.003055,0.003736,0.864570
AUGW,0.408856,-0.000986,0.068645,-0.720189
DECT,0.412054,-0.004898,0.116859,-0.991837
DECW,0.263351,-0.002896,0.123642,-0.769010
FEBT,0.501798,-0.004198,0.079271,-1.062882
